In [6]:
# pip install gensim

In [7]:
# ====================== 1. Import Library ======================
import pandas as pd
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from gensim import corpora, models

# ====================== 2. Load Dataset ======================
df = pd.read_csv("bbc_cleaned.csv")

# Pastikan kolom 'description' tidak kosong
df = df.dropna(subset=['description'])

# ====================== 3. Preprocessing ======================
def preprocess(text):
    return [word for word in simple_preprocess(text, deacc=True) if word not in STOPWORDS]

df['tokens'] = df['description'].apply(preprocess)

# ====================== 4. Dictionary dan Corpus ======================
dictionary = corpora.Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# ====================== 5. LDA Topic Modeling ======================
lda_model = models.LdaModel(corpus=corpus,
                            id2word=dictionary,
                            num_topics=5,         # Ubah jumlah topik jika diperlukan
                            random_state=42,
                            passes=15,
                            alpha='auto',
                            per_word_topics=True)

# ====================== 6. Tampilkan Topik ======================
topics = lda_model.print_topics(num_words=5)
for idx, topic in topics:
    print(f"Topik {idx + 1}:\n{topic}\n")


Topik 1:
0.005*"people" + 0.005*"year" + 0.004*"sir" + 0.004*"uk" + 0.004*"military"

Topik 2:
0.019*"says" + 0.009*"bbc" + 0.006*"south" + 0.006*"police" + 0.005*"government"

Topik 3:
0.011*"new" + 0.007*"deal" + 0.006*"uk" + 0.006*"trump" + 0.006*"white"

Topik 4:
0.009*"uk" + 0.009*"bbc" + 0.008*"england" + 0.007*"says" + 0.006*"year"

Topik 5:
0.008*"new" + 0.008*"bbc" + 0.007*"says" + 0.007*"people" + 0.006*"site"



In [8]:
# ====================== 7. Hitung Persentase Term per Topik ======================
def get_term_percentage(lda_model, corpus, dictionary, num_topics, num_words=5):
    # Inisialisasi dictionary untuk menyimpan bobot term per topik
    term_weights = {i: {} for i in range(num_topics)}
    
    # Ambil bobot term dari model LDA
    for topic_id in range(num_topics):
        topic_terms = lda_model.show_topic(topic_id, topn=num_words)
        for term, weight in topic_terms:
            term_weights[topic_id][term] = weight
    
    # Hitung total bobot per topik
    total_weights = {i: sum(term_weights[i].values()) for i in range(num_topics)}
    
    # Hitung persentase kontribusi term per topik
    term_percentage = {i: {} for i in range(num_topics)}
    for topic_id in range(num_topics):
        for term, weight in term_weights[topic_id].items():
            term_percentage[topic_id][term] = (weight / total_weights[topic_id]) * 100
    
    return term_percentage

# ====================== 8. Tampilkan Persentase Term ======================
term_percentage = get_term_percentage(lda_model, corpus, dictionary, num_topics=5, num_words=5)

for topic_id in range(5):
    print(f"\nPersentase Term untuk Topik {topic_id + 1}:")
    for term, percentage in term_percentage[topic_id].items():
        print(f"{term}: {percentage:.2f}%")


Persentase Term untuk Topik 1:
people: 22.06%
year: 20.49%
sir: 19.16%
uk: 19.15%
military: 19.15%

Persentase Term untuk Topik 2:
says: 42.24%
bbc: 19.50%
south: 13.64%
police: 12.99%
government: 11.63%

Persentase Term untuk Topik 3:
new: 30.59%
deal: 20.78%
uk: 16.86%
trump: 15.89%
white: 15.88%

Persentase Term untuk Topik 4:
uk: 23.77%
bbc: 22.86%
england: 21.00%
says: 17.77%
year: 14.61%

Persentase Term untuk Topik 5:
new: 22.31%
bbc: 21.64%
says: 19.54%
people: 18.60%
site: 17.90%


In [9]:
# ====================== 9. Evaluasi Model LDA ======================
from gensim.models import CoherenceModel
import numpy as np

# 1. Perplexity
perplexity = lda_model.log_perplexity(corpus)
print(f"Perplexity: {np.exp2(-perplexity):.2f}")

# 2. Coherence Score
coherence_model = CoherenceModel(model=lda_model, texts=df['tokens'], dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score:.2f}")

# 3. Log Likelihood
log_likelihood = lda_model.log_perplexity(corpus) * sum(len(doc) for doc in corpus)
print(f"Log Likelihood: {log_likelihood:.2f}")

# 4. Topic Diversity
def topic_diversity(model, topn=10):
    topics = model.show_topics(num_topics=model.num_topics, num_words=topn, formatted=False)
    all_words = [word for topic in topics for word, _ in topic[1]]
    unique_words = set(all_words)
    return len(unique_words) / (model.num_topics * topn)

diversity = topic_diversity(lda_model)
print(f"Topic Diversity: {diversity:.2f}")


Perplexity: 218.71
Coherence Score: 0.55
Log Likelihood: -92893.76
Topic Diversity: 0.72


In [10]:
!ngrok config add-authtoken 2wwdJFwINjpCSNa4DknOUB9DfzM_3p6Zi1VWZr323oyzneU41

Authtoken saved to configuration file: C:\Users\Sabrina\AppData\Local/ngrok/ngrok.yml


In [11]:
# pip install plotly

In [15]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.models import CoherenceModel
import plotly.express as px
import joblib

# Konfigurasi halaman Streamlit
st.set_page_config(
    page_title="Dashboard LDA Topic Modeling",
    page_icon="📊",
)

# Judul aplikasi
st.title("LDA Topic Modeling Dashboard")
st.write("Analisis topik dari dataset BBC menggunakan model LDA.")

st.markdown("---")
st.subheader("Hasil Analisis Topik")

# ====================== 1. Load dan Persiapkan Data ======================
@st.cache_data
def load_data():
    data = pd.read_csv("bbc_cleaned.csv")
    data = data.dropna(subset=['description'])
    return data

data = load_data()

# ====================== 2. Preprocessing dan Model LDA ======================
@st.cache_resource
def train_lda_model():
    # Preprocessing
    def preprocess(text):
        return [word for word in simple_preprocess(text, deacc=True) if word not in STOPWORDS]

    data['tokens'] = data['description'].apply(preprocess)
    
    # Dictionary dan Corpus
    dictionary = corpora.Dictionary(data['tokens'])
    corpus = [dictionary.doc2bow(text) for text in data['tokens']]
    
    # Latih model LDA
    lda_model = models.LdaModel(corpus=corpus,
                                id2word=dictionary,
                                num_topics=5,
                                random_state=42,
                                passes=15,
                                alpha='auto',
                                per_word_topics=True)
    
    # Simpan model (opsional)
    joblib.dump(lda_model, 'lda_model.pkl')
    joblib.dump(dictionary, 'dictionary.pkl')
    return lda_model, dictionary, corpus, data

lda_model, dictionary, corpus, data = train_lda_model()

# ====================== 3. Fungsi untuk Persentase Term ======================
def get_term_percentage(lda_model, num_topics, num_words=5):
    term_weights = {i: {} for i in range(num_topics)}
    for topic_id in range(num_topics):
        topic_terms = lda_model.show_topic(topic_id, topn=num_words)
        for term, weight in topic_terms:
            term_weights[topic_id][term] = weight
    total_weights = {i: sum(term_weights[i].values()) for i in range(num_topics)}
    term_percentage = {i: {} for i in range(num_topics)}
    for topic_id in range(num_topics):
        for term, weight in term_weights[topic_id].items():
            term_percentage[topic_id][term] = (weight / total_weights[topic_id]) * 100
    return term_percentage

term_percentage = get_term_percentage(lda_model, num_topics=5, num_words=5)

# ====================== 4. Komponen Dashboard ======================
# Dropdown untuk memilih topik
st.subheader("Pilih Topik")
topic_id = st.selectbox("Pilih Topik untuk Ditampilkan:", [f"Topik {i+1}" for i in range(5)], index=0)
selected_topic = int(topic_id.split()[1]) - 1

# Tampilkan kata kunci topik
st.subheader("Kata Kunci Topik")
topic_terms = lda_model.show_topic(selected_topic, topn=5)
keywords = ", ".join([f"{term} ({weight:.3f})" for term, weight in topic_terms])
st.write(f"Kata kunci untuk {topic_id}: {keywords}")

# Tampilkan persentase term
st.subheader("Persentase Term per Topik")
terms = list(term_percentage[selected_topic].keys())
percentages = list(term_percentage[selected_topic].values())
for term, percentage in zip(terms, percentages):
    st.write(f"{term}: {percentage:.2f}%")

# Visualisasi persentase term
st.subheader("Visualisasi Persentase Term")
fig = px.bar(
    x=terms,
    y=percentages,
    labels={'x': 'Term', 'y': 'Persentase (%)'},
    title=f'Persentase Term pada {topic_id}',
    color=terms,
    color_discrete_sequence=px.colors.qualitative.Plotly
)
fig.update_layout(showlegend=False)
st.plotly_chart(fig, use_container_width=True)

# ====================== 5. Informasi Tambahan ======================
st.markdown("---")
st.subheader("Informasi Dataset")
st.write(f"Jumlah dokumen: {len(data)}")
st.write("Dataset: bbc_cleaned.csv")
st.write("Jumlah topik: 5")
st.write("Jumlah kata kunci per topik: 5")

# ====================== 6. Evaluasi Model LDA ======================
st.markdown("---")
st.subheader("📈 Evaluasi Model LDA")

# Hitung metrik evaluasi
perplexity = np.exp2(-lda_model.log_perplexity(corpus))
coherence_model = CoherenceModel(model=lda_model, texts=data['tokens'], dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
log_likelihood = lda_model.log_perplexity(corpus) * sum(len(doc) for doc in corpus)

# Fungsi untuk menghitung Topic Diversity
def topic_diversity(model, topn=10):
    topics = model.show_topics(num_topics=model.num_topics, num_words=topn, formatted=False)
    all_words = [word for topic in topics for word, _ in topic[1]]
    unique_words = set(all_words)
    return len(unique_words) / (model.num_topics * topn)

diversity = topic_diversity(lda_model)

# Tampilkan metrik evaluasi sebagai card
col1, col2 = st.columns(2)
with col1:
    st.metric("Perplexity", f"{perplexity:.2f}")
    st.metric("Coherence Score", f"{coherence_score:.2f}")
with col2:
    st.metric("Topic Diversity", f"{diversity:.2f}")
    st.metric("Log Likelihood", f"{log_likelihood:.2f}")


Overwriting app.py


In [13]:
from pyngrok import ngrok
import threading
import time
import os

In [14]:
def run_streamlit():
    os.system("streamlit run app.py --server.port 8501")

thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

time.sleep(5)

public_url = ngrok.connect(addr=8501)
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://8dbf-36-69-140-252.ngrok-free.app" -> "http://localhost:8501"
